In [1]:
import pyforest
import sys
sys.path.append('/Users/jayceepang/msse/ATOM_CODE/UTILS/')
from RF_functions import *
from GenUtils import *
from dataset import *
import os
import pickle
from imblearn.over_sampling import SMOTEN, ADASYN, SMOTE
from sklearn.metrics import accuracy_score, balanced_accuracy_score,precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score, confusion_matrix,matthews_corrcoef

from rdkit import Chem
from rdkit.Chem import AllChem

In [2]:
og_neks = ['NEK2_binding','NEK2_inhibition','NEK3_binding','NEK5_binding','NEK9_binding','NEK9_inhibition']
samplings =['none_scaled','UNDER','SMOTE','ADASYN'] 
new_neks=['NEK1', 'NEK11', 'NEK2', 'NEK3', 'NEK4', 'NEK5',
       'NEK6', 'NEK9']
RF_types = ['RF','RF_BCW','BRFC','BRFC_BCW']
og_datapath = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/datasets/80train_20test/featurized/'
results_dir='/Users/jayceepang/msse/ATOM_CODE/examples/holdout/holdout_results/'
datapath = '/Users/jayceepang/msse/ATOM_CODE/datasets/holdout_data/featurized/'
rf_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/results/RF_results/'
gp_path ='/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/results/GP_results/'
test_results=[]
final_cols=['model','NEK','strategy','feat_type','RF_type', 'cm','recall', 'specificity', 'accuracy', 'precision', 
                'f1', 'ROC_AUC', 'MCC', 'balanced_accuracy']
feat_cols_dropped =['NEK','compound_id','base_rdkit_smiles', 'subset', 'active'] 

In [7]:
test_results=[]
final_cols=['model','NEK','strategy','feat_type','RF_type', 'cm','recall', 'specificity', 'accuracy', 'precision', 
                'f1', 'ROC_AUC', 'MCC', 'balanced_accuracy']
for nek in new_neks: 
    for i in og_neks: 
        for samp in samplings: 
            trainX=pd.read_csv(f'{og_datapath}{i}_MFP_{samp}_trainX.csv')
            train_y =pd.read_csv(f'{og_datapath}{i}_MFP_{samp}_train_y.csv').to_numpy().reshape(-1)
            testX=pd.read_csv(f'{datapath}{nek}_MFP_none_unscaled.csv').drop(columns=feat_cols_dropped)
            test_y=pd.read_csv(f'{datapath}{nek}_MFP_none_unscaled.csv')['active'].to_numpy().reshape(-1)
            for rf in RF_types: 
                model_name = f'{i}_MFP_{samp}_{rf}'
                
                print(nek, model_name) 
                model = RF_functions.rf_models(trainX, train_y, testX, test_y, rf, {}, True)
                # train_df = gather_rf_results(model, trainX, train_y)
                test_df = gather_rf_results(model, testX, test_y)
                print()

                test_df['model'] = f'{nek}_{model_name}'
                test_df['NEK'] =nek
                test_df['feat_type'] = 'MFP'
                test_df['strategy'] = samp
                test_df['RF_type'] = rf
                test_df['prod_model'] = f'{i}_{samp}_GP_matern'
                test_df.to_csv(f'{results_dir}{nek}_{model_name}_holdout_test.csv',index=False) 
                test_results.append(test_df.iloc[[0]][final_cols].values.flatten())

    all_test =  pd.DataFrame(test_results,columns=final_cols)
    all_test['modeling_type'] = 'RF' 
    all_test['set'] = 'nanobret_cellular_holdout' 

    # all_test.to_csv(f'{results_dir}RF/RF_holdout_test_results_all_NEK.csv', index=False)                 
         

NEK1 NEK2_binding_MFP_none_scaled_RF
TRAIN: acc: 1.000, precision: 1.000, recall: 1.000, spec: 1.000
TEST: acc: 0.780, precision: 0.047, recall: 0.100, spec: 0.835
func:'rf_models'  took: 5.3459 sec

NEK1 NEK2_binding_MFP_none_scaled_RF_BCW
TRAIN: acc: 1.000, precision: 1.000, recall: 1.000, spec: 1.000
TEST: acc: 0.780, precision: 0.032, recall: 0.067, spec: 0.838
func:'rf_models'  took: 2.6180 sec

NEK1 NEK2_binding_MFP_none_scaled_BRFC


/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(


TRAIN: acc: 0.837, precision: 0.197, recall: 1.000, spec: 0.831
TEST: acc: 0.375, precision: 0.104, recall: 0.967, spec: 0.327
func:'rf_models'  took: 0.2710 sec

NEK1 NEK2_binding_MFP_none_scaled_BRFC_BCW


/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(


TRAIN: acc: 0.525, precision: 0.078, recall: 1.000, spec: 0.506
TEST: acc: 0.792, precision: 0.079, recall: 0.167, spec: 0.843
func:'rf_models'  took: 0.4693 sec

NEK1 NEK2_binding_MFP_UNDER_RF
TRAIN: acc: 1.000, precision: 1.000, recall: 1.000, spec: 1.000
TEST: acc: 0.420, precision: 0.068, recall: 0.533, spec: 0.411
func:'rf_models'  took: 0.1919 sec

NEK1 NEK2_binding_MFP_UNDER_RF_BCW
TRAIN: acc: 1.000, precision: 1.000, recall: 1.000, spec: 1.000
TEST: acc: 0.407, precision: 0.067, recall: 0.533, spec: 0.397
func:'rf_models'  took: 0.1974 sec

NEK1 NEK2_binding_MFP_UNDER_BRFC


...

TRAIN: acc: 0.998, precision: 0.996, recall: 1.000, spec: 0.996
TEST: acc: 0.833, precision: 0.000, recall: 0.000, spec: 0.895
func:'rf_models'  took: 0.9222 sec

NEK3 NEK9_inhibition_MFP_ADASYN_BRFC_BCW


/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(


TRAIN: acc: 0.998, precision: 0.996, recall: 1.000, spec: 0.996
TEST: acc: 0.843, precision: 0.000, recall: 0.000, spec: 0.906
func:'rf_models'  took: 0.9629 sec

NEK4 NEK2_binding_MFP_none_scaled_RF


/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [17]:
all_test=pd.read_csv(f'{results_dir}RF/RF_holdout_test_results_all_NEK.csv')
# all_test['prod_model'] = all_test['model'].astype(str).apply(lambda x: '_'.join(x.split('_')[2:]))
all_test['prod_model'] = all_test['model'].astype(str).apply(lambda x: '_'.join(x.split('_')[1:]))
all_test

,model,NEK,strategy,feat_type,RF_type,cm,recall,specificity,accuracy,precision,f1,ROC_AUC,MCC,balanced_accuracy,modeling_type,set,prod_model
0,NEK1_NEK2_binding_MFP_none_scaled_RF,NEK1,none_scaled,MFP,RF,"[309, 61, 27, 3]",0.100000,0.835135,0.7800,0.046875,0.063830,0.467568,-0.046603,0.467568,RF,nanobret_cellular_holdout,NEK2_binding_MFP_none_scaled_RF
1,NEK1_NEK2_binding_MFP_none_scaled_RF_BCW,NEK1,none_scaled,MFP,RF_BCW,"[310, 60, 28, 2]",0.066667,0.837838,0.7800,0.032258,0.043478,0.452252,-0.069501,0.452252,RF,nanobret_cellular_holdout,NEK2_binding_MFP_none_scaled_RF_BCW
2,NEK1_NEK2_binding_MFP_none_scaled_BRFC,NEK1,none_scaled,MFP,BRFC,"[121, 249, 1, 29]",0.966667,0.327027,0.3750,0.104317,0.188312,0.646847,0.168017,0.646847,RF,nanobret_cellular_holdout,NEK2_binding_MFP_none_scaled_BRFC
3,NEK1_NEK2_binding_MFP_none_scaled_BRFC_BCW,NEK1,none_scaled,MFP,BRFC_BCW,"[312, 58, 25, 5]",0.166667,0.843243,0.7925,0.079365,0.107527,0.504955,0.007165,0.504955,RF,nanobret_cellular_holdout,NEK2_binding_MFP_none_scaled_BRFC_BCW
4,NEK1_NEK2_binding_MFP_UNDER_RF,NEK1,UNDER,MFP,RF,"[152, 218, 14, 16]",0.533333,0.410811,0.4200,0.068376,0.121212,0.472072,-0.029859,0.472072,RF,nanobret_cellular_holdout,NEK2_binding_MFP_UNDER_RF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,NEK3_NEK9_inhibition_MFP_SMOTE_BRFC_BCW,NEK3,SMOTE,MFP,BRFC_BCW,"[301, 71, 13, 15]",0.535714,0.809140,0.7900,0.174419,0.263158,0.672427,0.214176,0.672427,RF,nanobret_cellular_holdout,NEK9_inhibition_MFP_SMOTE_BRFC_BCW
380,NEK3_NEK9_inhibition_MFP_ADASYN_RF,NEK3,ADASYN,MFP,RF,"[333, 39, 28, 0]",0.000000,0.895161,0.8325,0.000000,0.000000,0.447581,-0.090175,0.447581,RF,nanobret_cellular_holdout,NEK9_inhibition_MFP_ADASYN_RF
381,NEK3_NEK9_inhibition_MFP_ADASYN_RF_BCW,NEK3,ADASYN,MFP,RF_BCW,"[338, 34, 28, 0]",0.000000,0.908602,0.8450,0.000000,0.000000,0.454301,-0.083619,0.454301,RF,nanobret_cellular_holdout,NEK9_inhibition_MFP_ADASYN_RF_BCW
382,NEK3_NEK9_inhibition_MFP_ADASYN_BRFC,NEK3,ADASYN,MFP,BRFC,"[333, 39, 28, 0]",0.000000,0.895161,0.8325,0.000000,0.000000,0.447581,-0.090175,0.447581,RF,nanobret_cellular_holdout,NEK9_inhibition_MFP_ADASYN_BRFC


In [20]:
# all_test['data NEK'] = all_test['NEK'] 
all_test['model NEK'] = all_test['prod_model'].astype(str).str.split('_').str[0:2].apply(lambda x: '_'.join(x))
all_test=all_test.rename(columns={'NEK':'data NEK','strategy': 'model strategy', 'RF_type':'model version' })
new_order = ['model', 'data NEK', 'prod_model', 'model NEK', 'model version','model strategy', 'feat_type', 'cm', 'recall',
             'specificity', 'accuracy', 'precision', 'f1', 'ROC_AUC', 'MCC', 'balanced_accuracy',
             'set', 'modeling_type']
all_test = all_test[new_order]
all_test.to_csv(f'{results_dir}RF/RF_nanobret_cellular_holdout_results.csv', index=False)

In [19]:
all_test

,model,data NEK,prod_model,model NEK,model version,model strategy,feat_type,cm,recall,specificity,accuracy,precision,f1,ROC_AUC,MCC,balanced_accuracy,set,modeling_type
0,NEK1_NEK2_binding_MFP_none_scaled_RF,NEK1,NEK2_binding_MFP_none_scaled_RF,NEK2_binding,RF,none_scaled,MFP,"[309, 61, 27, 3]",0.100000,0.835135,0.7800,0.046875,0.063830,0.467568,-0.046603,0.467568,nanobret_cellular_holdout,RF
1,NEK1_NEK2_binding_MFP_none_scaled_RF_BCW,NEK1,NEK2_binding_MFP_none_scaled_RF_BCW,NEK2_binding,RF_BCW,none_scaled,MFP,"[310, 60, 28, 2]",0.066667,0.837838,0.7800,0.032258,0.043478,0.452252,-0.069501,0.452252,nanobret_cellular_holdout,RF
2,NEK1_NEK2_binding_MFP_none_scaled_BRFC,NEK1,NEK2_binding_MFP_none_scaled_BRFC,NEK2_binding,BRFC,none_scaled,MFP,"[121, 249, 1, 29]",0.966667,0.327027,0.3750,0.104317,0.188312,0.646847,0.168017,0.646847,nanobret_cellular_holdout,RF
3,NEK1_NEK2_binding_MFP_none_scaled_BRFC_BCW,NEK1,NEK2_binding_MFP_none_scaled_BRFC_BCW,NEK2_binding,BRFC_BCW,none_scaled,MFP,"[312, 58, 25, 5]",0.166667,0.843243,0.7925,0.079365,0.107527,0.504955,0.007165,0.504955,nanobret_cellular_holdout,RF
4,NEK1_NEK2_binding_MFP_UNDER_RF,NEK1,NEK2_binding_MFP_UNDER_RF,NEK2_binding,RF,UNDER,MFP,"[152, 218, 14, 16]",0.533333,0.410811,0.4200,0.068376,0.121212,0.472072,-0.029859,0.472072,nanobret_cellular_holdout,RF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,NEK3_NEK9_inhibition_MFP_SMOTE_BRFC_BCW,NEK3,NEK9_inhibition_MFP_SMOTE_BRFC_BCW,NEK9_inhibition,BRFC_BCW,SMOTE,MFP,"[301, 71, 13, 15]",0.535714,0.809140,0.7900,0.174419,0.263158,0.672427,0.214176,0.672427,nanobret_cellular_holdout,RF
380,NEK3_NEK9_inhibition_MFP_ADASYN_RF,NEK3,NEK9_inhibition_MFP_ADASYN_RF,NEK9_inhibition,RF,ADASYN,MFP,"[333, 39, 28, 0]",0.000000,0.895161,0.8325,0.000000,0.000000,0.447581,-0.090175,0.447581,nanobret_cellular_holdout,RF
381,NEK3_NEK9_inhibition_MFP_ADASYN_RF_BCW,NEK3,NEK9_inhibition_MFP_ADASYN_RF_BCW,NEK9_inhibition,RF_BCW,ADASYN,MFP,"[338, 34, 28, 0]",0.000000,0.908602,0.8450,0.000000,0.000000,0.454301,-0.083619,0.454301,nanobret_cellular_holdout,RF
382,NEK3_NEK9_inhibition_MFP_ADASYN_BRFC,NEK3,NEK9_inhibition_MFP_ADASYN_BRFC,NEK9_inhibition,BRFC,ADASYN,MFP,"[333, 39, 28, 0]",0.000000,0.895161,0.8325,0.000000,0.000000,0.447581,-0.090175,0.447581,nanobret_cellular_holdout,RF
